
# Assignment No 3b
###### *Sibt ul Hussain*
----
## Goal

Your goal in this part of assigment is to implement a Decision Tree Classifier for categorical variables.

**Note** Please note that you are allowed to use only those libraries which we have discussed in the class, i.e. numpy, scipy, pandas.

## Submission Instructions
You are required to submit the original notebook file on the Slate (with .ipynb extension), with complete set of outputs. Students failing to do so will get zero marks. 

*Please read each step carefully and understand it fully before proceeding with code writing*

## Plagiarism
Any form of plagiarism will not be tolerated and result in 0 marks.



### Decision Tree Classifier

Now in this assignment we will be implementing the Decision Classifier for both Continuous and Categorical attributes.

Decision tree can be built by using any of the following split criterias, namely:
 - Information Gain
 - Gini Index
 - CART 

However, you are required here to implement the decision tree with information gain as splitting criterion.

Remember in my code i am not looking for maximizing the information gain, instead i am looking for minimizing the split entropy. Recall,
$$Information Gain  = H(D) - H(D_Y,D_N)$$

Where,

$H(D)$ is the data set entroy and $H(D_Y,D_N)$ is split entropy. Since $H(D)$ is constant for the given dataset so maximizing the entropy is equal to minimizing the split entropy and that is what is being represented in my code outputs.

In [1]:
%pylab inline
import scipy.stats
from collections import defaultdict  # default dictionary 

Populating the interactive namespace from numpy and matplotlib


In [2]:
#Cutomize the Matplotlib for beautiful plots...
# import dmStyle
# dmStyle.customize_mpl()


In [3]:
import pandas as pd
import tools as t # set of tools for plotting, data splitting, etc..

In [4]:
def getSplits(categories):
    '''
        function returns list of splits for the given list of categorical variables...
        
        Input:
        ------------
            categories: a list of unique categories...
        
        Return:
        ------------
            list of splits(tuples) for given list of categorical variables. Each pair of sublists
            defines the left and right splits, e.g. This list
            [('y', 'f'), ('s', 'g'), ('f'), ('y', 's', 'g')]
            
            defines two splits with each pair representing a different split.
        Examples:
        ------------
        splits=getSplits(['a1','a2','a3','a4']) will return 
        [('a1', 'a2', 'a4'), ('a3',), ('a2', 'a4'), 
        ('a1', 'a3'), ('a1', 'a4'), ('a3', 'a2'), ('a1', 'a3', 'a2'), 
        ('a4',), ('a3', 'a4'), ('a1', 'a2'), ('a1',), ('a3', 'a2', 'a4'), ('a2',), ('a1', 'a3', 'a4')]

            
    '''
    categories=set(categories)
    tsplits=t.get_powerset(categories,len(categories)-1) # get all the power sets with the given cardinality...
    flist=[]
    for s in tsplits:

        if not s in flist:
            r=categories.difference(s)
    #         print s, tuple(r)
            flist.append(s)
            flist.append(r)

    olist=[]

    for s in flist:
        ilist=[]

        for k in s:
            ilist.append(k)
        olist.append(tuple(ilist))

#     print olist
    
    return olist

In [5]:
splits=getSplits(['a1','a2','a3','a4'])
print splits,len(splits)

[('a1', 'a2', 'a4'), ('a3',), ('a2', 'a4'), ('a1', 'a3'), ('a1', 'a4'), ('a3', 'a2'), ('a1', 'a3', 'a2'), ('a4',), ('a3', 'a4'), ('a1', 'a2'), ('a1',), ('a3', 'a2', 'a4'), ('a2',), ('a1', 'a3', 'a4')] 14


In [6]:
class Node:
    def __init__(self,purity,klasslabel='',score=0,split=[],fidx=-1):
        '''
            purity: purity level at which to stop
            klasslabel: klasslabel of the node, (for leaf node)
            score: information gain of the newly added node
            split: splitting threshold
            fidx: feature index            
        '''
        self.lchild=None
        self.rchild=None
        self.klasslabel=klasslabel        
        self.split=split
        self.score=score
        self.fidx=fidx
        self.purity=purity
        self.ftype= 'categorical' if type(self.split) in [tuple, str, numpy.string_] else 'continuous' # feature type 

        
    def set_childs(self,lchild,rchild):
        # YOUR CODE HERE
        self.lchild = lchild 
        self.rchild = rchild 
        #raise NotImplementedError()
    def isleaf(self):
        # YOUR CODE HERE
        return self.lchild == None and self.rchild == None
        #raise NotImplementedError()
    def isless_than_eq(self, X):
        # YOUR CODE HERE
        leftidx = np.nonzero(X[:, fidx] <= self.split)
        rightidx = np.nonzero(X[:, fidx] > self.split)
        return X[leftidx, :], X[rightidx, :]
        #raise NotImplementedError()
    def get_str(self):        
        if self.isleaf():
            return 'C(class={},Purity={})'.format(self.klasslabel,self.purity)
        else:
            return 'I(Fidx={},Score={},Split={})'.format(self.fidx,self.score,self.split)
    

In [7]:
# A placeholder class 
# TODO: You have to implement the following class, remember from the lectures that you will 
# need to build a model for each different class you are trying to identify...

In [9]:
### import pdb
## Your code goes here...
# You might need to define auxliary classes for composition.. ?
class DecisionTree:
    ''' Implements the Decision Tree For Classification... '''
    def __init__(self, purityp, exthreshold, maxdepth=10):        
        self.purity = purityp
        self.exthreshold = exthreshold
        self.maxdepth = maxdepth
        
        pass
    def train(self, X, Y):
        ''' Train Decision Tree using the given 
            X [m x d] data matrix and Y labels matrix
            
            Input:
            ------
            X: [m x d] a data matrix of m d-dimensional examples.
            Y: [m x 1] a label vector.
            
            Returns:
            -----------
            Nothing
            '''
        nexamples, nfeatures = X.shape
        # # now go and train a model for each class...
        # YOUR CODE HERE
        self.classes = np.unique(Y)
        self.tree = self.build_tree(X, Y, self.maxdepth)
        #raise NotImplementedError()
        
    def getPurityResult(self, Y):
        classes = np.unique(Y)
        acc = np.zeros_like(classes)
        for i, c in enumerate(classes): 
            acc[i] = np.sum(Y==c)
        acc = acc / (1. * np.sum(acc))
        return acc.max(), classes[acc.argmax()]
    
    def build_tree(self, X, Y, depth):
        """ 
            Function is used to recursively build the decision Tree 
          
            Input
            -----
            X: [m x d] a data matrix of m d-dimensional examples.
            Y: [m x 1] a label vector.
            
            Returns
            -------
            root node of the built tree...
        """
        nexamples, nfeatures = X.shape
        
        #print 'function call =', len(Y)
        # YOUR CODE HERE
        #print '\n\nNew node'
        #raise NotImplementedError()
        return node
        
        
    def test(self, X):
        
        ''' Test the trained classifiers on the given set of examples 
        
                   
            Input:
            ------
            X: [m x d] a data matrix of m d-dimensional test examples.
           
            Returns:
            -----------
                pclass: the predicted class for each example, i.e. to which it belongs
        '''
        
        nexamples, nfeatures = X.shape
        pclasses = self.predict(X)
        
        # your code go here...
        
        return np.array(pclasses)
    
    def generateCounts(self, labels, numclasses):
        out = np.zeros((numclasses,))
        for i, c in enumerate(np.unique(labels)):
            out[i] = np.sum(labels == c)
        return out 
    
    def calculateEntropy(self, D):
        Dprobs = D / (1. * np.sum(D))+1e-16
        out = np.sum(Dprobs * np.log(Dprobs)) * -1
        return out
    
    def calculateSplitEntropy(self, Dy, Dn):
        nexamples = np.sum(Dy+Dn) * 1.
        a = (np.sum(Dy)/nexamples)*(self.calculateEntropy(Dy))
        b = (np.sum(Dn)/nexamples)*(self.calculateEntropy(Dn))
        return a+b
    
    def evaluate_categorical_attribute(self, feat, Y):
            """
            Evaluates the cateogrical attribute for all possible split points, i.e. 2^(m-1)-2 for
            possible feature selection

            Input:
            ---------
            feat: a categorical feature
            Y: labels

            Returns:
            ----------
            v: splitting threshold
            score: splitting score
            Xlidx: Index of examples belonging to left child node
            Xridx: Index of examples belonging to right child node

            """
            categories = set(feat)
            splits = getSplits(categories) if len(categories) > 1 else tuple(categories)
            freq = scipy.stats.itemfreq(Y)
            nclasses = len(self.classes)
            
            # YOUR CODE HERE
            splitscores = np.zeros((len(splits)))
            
            for i, splitpoint in enumerate(splits):
                splitpoint = np.asarray(splitpoint)
                Dy = Y[np.where(np.in1d(feat, splitpoint))]
                Dn = Y[np.where(np.in1d(feat, splitpoint, invert=True))]
                
                DyClassCounts = self.generateCounts(Dy, nclasses)
                DnClassCounts = self.generateCounts(Dn, nclasses)
                
                splitscores[i] = self.calculateSplitEntropy(DyClassCounts, DnClassCounts)
            
            csplit = splits[splitscores.argmin()]
            score = splitscores.min()
            Xlidx = np.nonzero(np.in1d(feat, csplit))
            Xridx = np.nonzero(np.in1d(feat, csplit, invert=True))
            
            return csplit, score, Xlidx, Xridx
        
    def evaluate_numerical_attribute(self, feat, Y):
        '''
            Evaluates the numerical attribute for all possible split points for
            possible feature selection
            
            Input:
            ---------
            feat: a contiuous feature
            Y: labels
            
            Returns:
            ----------
            v: splitting threshold
            score: splitting score
            Xlidx: Index of examples belonging to left child node
            Xridx: Index of examples belonging to right child node
            
        '''
        
        classes = np.unique(Y)
        nclasses = len(self.classes)
        f = np.sort(feat)
        
        # YOUR CODE HERE
        YclassCounts = self.generateCounts(Y, nclasses)
        
        points = np.sort(np.unique(feat)) 
        midpoints = (points[:-1]+points[1:]) / 2.
        splitscores = np.zeros((midpoints.shape[0],))
        
        for i, splitpoint in enumerate(midpoints):
            DyClassCounts = self.generateCounts(Y[feat <= splitpoint], nclasses)
            DnClassCounts = YclassCounts - DyClassCounts
            splitscores[i] = self.calculateSplitEntropy(DyClassCounts, DnClassCounts)
            
        v = midpoints[splitscores.argmin()]
        score = splitscores.min()
        Xlidx = np.nonzero(feat <= v)
        Xridx = np.nonzero(feat > v)
        
        return v, score, Xlidx, Xridx
    
    def predict(self, X):
        
        """
        Test the trained classifiers on the given example X
        
                   
            Input:
            ------
            X: [1 x d] a d-dimensional test example.
           
            Returns:
            -----------
                pclass: the predicted class for the given example, i.e. to which it belongs
        """
        z = []
        
        for idx in range(X.shape[0]):
            
            z.append(self._predict(self.tree, X[idx, :]))
        
        return z 
    
    def _predict(self, node, X):
        # YOUR CODE HERE
        if node.isleaf(): 
            return node.klasslabel 
        else:
            return self._predict(node.lchild, X) if (X[node.fidx] in node.split) else self._predict(node.rchild, X)
        #raise NotImplementedError()
        
    def __str__(self):
        str = '---------------------------------------------------'
        str += '\n A Decision Tree With Depth={}'.format(self.find_depth())
        str += self.__print(self.tree)
        str += '\n---------------------------------------------------'
        return str  # self.__print(self.tree)        
        
     
   
    def find_depth(self):
        return self._find_depth(self.tree)
    def _find_depth(self, node):
        if not node:
            return
        if node.isleaf():
            return 1
        else:
            return max(self._find_depth(node.lchild), self._find_depth(node.rchild)) + 1
    def __print(self, node, depth=0):
        
        ret = ""

        # Print right branch
        if node.rchild:
            ret += self.__print(node.rchild, depth + 1)

        # Print own value
        
        ret += "\n" + ("    "*depth) + node.get_str()

        # Print left branch
        if node.lchild:
            ret += self.__print(node.lchild, depth + 1)
        
        return ret

# Lets test our code for the given example in the book.

In [10]:
#load the Iris dataset
tdata=pd.read_csv('./iris.data')
tdata.columns=['SepalLength','SepalWidth','PetalLength','PetalWidth','Class']

In [11]:
tx=tdata['SepalLength'].dropna()
tx[(tdata['SepalLength']>=4.3) & (tdata['SepalLength']<=5.2)]='a1'
tx[(tdata['SepalLength']>5.2) & (tdata['SepalLength']<=6.1)]='a2'
tx[(tdata['SepalLength']>6.1) & (tdata['SepalLength']<=7.0)]='a3'
tx[(tdata['SepalLength']>7.0) & (tdata['SepalLength']<=7.9)]='a4'

In [12]:
print tx.values
Y=tdata['Class'].dropna()
Y[Y=='Iris-virginica']='Iris-versicolor'
Y=Y.values
print Y

['a1' 'a1' 'a1' 'a1' 'a2' 'a1' 'a1' 'a1' 'a1' 'a2' 'a1' 'a1' 'a1' 'a2' 'a2'
 'a2' 'a1' 'a2' 'a1' 'a2' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1'
 'a2' 'a1' 'a2' 'a1' 'a1' 'a2' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1'
 'a1' 'a1' 'a2' 'a1' 'a3' 'a3' 'a3' 'a2' 'a3' 'a2' 'a3' 'a1' 'a3' 'a1' 'a1'
 'a2' 'a2' 'a2' 'a2' 'a3' 'a2' 'a2' 'a3' 'a2' 'a2' 'a2' 'a3' 'a2' 'a3' 'a3'
 'a3' 'a3' 'a2' 'a2' 'a2' 'a2' 'a2' 'a2' 'a2' 'a2' 'a3' 'a3' 'a2' 'a2' 'a2'
 'a2' 'a2' 'a1' 'a2' 'a2' 'a2' 'a3' 'a1' 'a2' 'a3' 'a2' 'a4' 'a3' 'a3' 'a4'
 'a1' 'a4' 'a3' 'a4' 'a3' 'a3' 'a3' 'a2' 'a2' 'a3' 'a3' 'a4' 'a4' 'a2' 'a3'
 'a2' 'a4' 'a3' 'a3' 'a4' 'a3' 'a2' 'a3' 'a4' 'a4' 'a4' 'a3' 'a3' 'a2' 'a4'
 'a3' 'a3' 'a2' 'a3' 'a3' 'a3' 'a2' 'a3' 'a3' 'a3' 'a3' 'a3' 'a3' 'a2']
['Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'I

In [13]:
%pdb off
dt=DecisionTree(0.95,5,5)
dt.classes = np.unique(Y)
split, gain, Xlidx, Xridx = dt.evaluate_categorical_attribute(tx,Y)
print 
print split, gain

Automatic pdb calling has been turned OFF

('a1',) 0.353991894227


In [16]:
from nose.tools import assert_almost_equal, assert_almost_equals

%pdb off
dt=DecisionTree(0.95,5,5)
dt.classes = np.unique(Y)
split, gain, Xlidx, Xridx = dt.evaluate_categorical_attribute(tx,Y)

print len(split)
print split[0]
assert_almost_equal('a1', split[0])
assert_almost_equal(gain, 0.51, 1)

Automatic pdb calling has been turned OFF
1
a1


AssertionError: 0.35399189422691291 != 0.51 within 1 places

# Now lets test on [Mushroom](https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.names) dataset


This data set includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family (pp. 500-525).  Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended.  This latter class was combined with the poisonous one.  The Guide clearly states that there is no simple rule for determining the edibility of a mushroom; no rule like ``leaflets three, let it be'' for Poisonous Oak and Ivy.


- Number of Instances: 8124

- Number of Attributes: 22 (all nominally valued)

- Attribute Information: (classes: edible=e, poisonous=p)
         1. cap-shape:                bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s
         2. cap-surface:              fibrous=f,grooves=g,scaly=y,smooth=s
         3. cap-color:                brown=n,buff=b,cinnamon=c,gray=g,green=r, pink=p,purple=u,red=e,white=w,yellow=y
         4. bruises?:                 bruises=t,no=f
         5. odor:                     almond=a,anise=l,creosote=c,fishy=y,foul=f, musty=m,none=n,pungent=p,spicy=s
         6. gill-attachment:          attached=a,descending=d,free=f,notched=n
         7. gill-spacing:             close=c,crowded=w,distant=d
         8. gill-size:                broad=b,narrow=n
         9. gill-color:               black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y
        10. stalk-shape:              enlarging=e,tapering=t
        11. stalk-root:               bulbous=b,club=c,cup=u,equal=e, rhizomorphs=z,rooted=r,missing=?
        12. stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
        13. stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
        14. stalk-color-above-ring:   brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
        15. stalk-color-below-ring:   brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
        16. veil-type:                partial=p,universal=u
        17. veil-color:               brown=n,orange=o,white=w,yellow=y
        18. ring-number:              none=n,one=o,two=t
        19. ring-type:                cobwebby=c,evanescent=e,flaring=f,large=l, none=n,pendant=p,sheathing=s,zone=z
        20. spore-print-color:        black=k,brown=n,buff=b,chocolate=h,green=r, orange=o,purple=u,white=w,yellow=y
        21. population:               abundant=a,clustered=c,numerous=n, scattered=s,several=v,solitary=y
        22. habitat:                  grasses=g,leaves=l,meadows=m,paths=p, urban=u,waste=w,woods=d

- Missing Attribute Values: 2480 of them (denoted by "?"), all for  attribute #11.

- Class Distribution: 
        --    edible: 4208 (51.8%)
        -- poisonous: 3916 (48.2%)
        --     total: 8124 instances

In [20]:
#load the data set
data=pd.read_csv('./mushrooms.csv')
data.columns=columns=['class', 'cap-shape','cap-surface','cap-color','bruises?','odor','gill-attachment','gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat']
print data.describe()

       class cap-shape cap-surface cap-color bruises?  odor gill-attachment  \
count   8124      8124        8124      8124     8124  8124            8124   
unique     2         6           4        10        2     9               2   
top        e         x           y         n        f     n               f   
freq    4208      3656        3244      2284     4748  3528            7914   

       gill-spacing gill-size gill-color   ...   stalk-surface-below-ring  \
count          8124      8124       8124   ...                       8124   
unique            2         2         12   ...                          4   
top               c         b          b   ...                          s   
freq           6812      5612       1728   ...                       4936   

       stalk-color-above-ring stalk-color-below-ring veil-type veil-color  \
count                    8124                   8124      8124       8124   
unique                      9                      9         1  

In [21]:
data.tail()

,class,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l
8123,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,o,c,l


In [22]:
# Get your data in matrix
X=np.asarray(data.ix[:,1:].dropna())
Y=np.asarray(data.ix[:,0].dropna())
print " Data Set Dimensions=", X.shape, " True Class labels dimensions", Y.shape   


 Data Set Dimensions= (8124, 22)  True Class labels dimensions (8124,)


In [23]:
# Split your data into training and test-set... 
# see the documentation of split_data in tools for further information...
Xtrain,Ytrain,Xtest,Ytest=t.split_data(X,Y)

print " Training Data Set Dimensions=", Xtrain.shape, "Training True Class labels dimensions", Ytrain.shape   
print " Test Data Set Dimensions=", Xtest.shape, "Test True Class labels dimensions", Ytrain.shape   


 Training Data Set Dimensions= (5687, 22) Training True Class labels dimensions (5687,)
 Test Data Set Dimensions= (2437, 22) Test True Class labels dimensions (5687,)


In [62]:
np.unique(Xtrain[:, 0])

array(['b', 'f', 's', 'x'], dtype=object)

In [76]:
# Lets train a Decision Tree Classifier two features
%pdb
feat=np.arange(2)
dt=DecisionTree(0.95,5,10)
dt.train(Xtrain[:,feat],Ytrain)

Automatic pdb calling has been turned ON


In [77]:
print dt

---------------------------------------------------
 A Decision Tree With Depth=5
            C(class=Iris-versicolor,Purity=1.0)
        I(Fidx=0,Score=0.588907513006,Split=('x', 'b'))
                C(class=Iris-versicolor,Purity=0.586206896552)
            I(Fidx=0,Score=0.669596843712,Split=('b',))
                C(class=Iris-versicolor,Purity=0.642857142857)
    I(Fidx=0,Score=0.589340048175,Split=('x', 'b'))
            C(class=Iris-versicolor,Purity=1.0)
        I(Fidx=0,Score=0.588649329276,Split=('x', 'b'))
                C(class=Iris-versicolor,Purity=0.684210526316)
            I(Fidx=0,Score=0.649079083817,Split=('b',))
                C(class=Iris-versicolor,Purity=0.588235294118)
I(Fidx=0,Score=0.582717275383,Split=('f',))
        C(class=Iris-versicolor,Purity=1.0)
    I(Fidx=0,Score=0.5349428671,Split=('x', 's'))
            C(class=Iris-versicolor,Purity=0.625)
        I(Fidx=0,Score=0.599578309458,Split=('s',))
            C(class=Iris-versicolor,Purity=0.8)
------

In [78]:
pclasses=dt.predict(Xtest[:,feat])
#Lets see how good we are doing, by finding the accuracy on the test set..
print pclasses
print np.sum(pclasses==Ytest)
print "Accuracy = ", np.sum(pclasses==Ytest)/float(Ytest.shape[0])

['Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor']
30
Accuracy =  0.666666666667


In [79]:
from nose.tools import assert_greater_equal
Xtrain,Ytrain,Xtest,Ytest=t.split_data(X,Y)
feat=np.arange(2)
dt=DecisionTree(0.95,5,10)
dt.train(Xtrain[:,feat],Ytrain)


pclasses=dt.predict(Xtest[:,feat])
acc = np.sum(pclasses==Ytest)/float(Ytest.shape[0])
print acc
assert_greater_equal(acc, 0.60)

0.666666666667


In [ ]:
from nose.tools import assert_greater_equal
Xtrain,Ytrain,Xtest,Ytest=t.split_data(X,Y)

dt=DecisionTree(0.95,5,10)
dt.train(Xtrain,Ytrain)

pclasses=dt.predict(Xtest)
acc = np.sum(pclasses==Ytest)/float(Ytest.shape[0])

assert_greater_equal(acc, 0.90)

AssertionError: 0.62222222222222223 not greater than or equal to 0.9

> /home/muneebaadil/anaconda2/lib/python2.7/unittest/case.py(410)fail()
    408     def fail(self, msg=None):
    409         """Fail immediately, with the given message."""
--> 410         raise self.failureException(msg)
    411 
    412     def assertFalse(self, expr, msg=None):



# Lets Train on all the features and for both the classes....

In [56]:
# Split your data into training and test-set... 
# see the documentation of split_data in tools for further information...
Xtrain,Ytrain,Xtest,Ytest=t.split_data(X,Y)

print " Training Data Set Dimensions=", Xtrain.shape, "Training True Class labels dimensions", Ytrain.shape   
print " Test Data Set Dimensions=", Xtest.shape, "Test True Class labels dimensions", Ytrain.shape   


 Training Data Set Dimensions= (104, 22) Training True Class labels dimensions (104,)
 Test Data Set Dimensions= (45, 22) Test True Class labels dimensions (104,)


In [57]:
feat=arange(22)#[0, 1, 2, 3]
dt=DecisionTree(0.95,5,10)
dt.train(Xtrain[:,feat],Ytrain)
pclasses=dt.predict(Xtest[:,feat])
#Lets see how good we are doing, by finding the accuracy on the test set..
print np.sum(pclasses==Ytest)
print "Accuracy = ", np.sum(pclasses==Ytest)/float(Ytest.shape[0])

16
Accuracy =  0.355555555556


In [58]:
print dt

---------------------------------------------------
 A Decision Tree With Depth=11
                        C(class=Iris-versicolor,Purity=1.0)
                    I(Fidx=1,Score=0.449868115695,Split=('s',))
                        C(class=Iris-setosa,Purity=0.5)
                I(Fidx=1,Score=0.480722619292,Split=('s',))
                    C(class=Iris-setosa,Purity=1.0)
            I(Fidx=4,Score=0.655229480115,Split=('l',))
                    C(class=Iris-setosa,Purity=0.75)
                I(Fidx=4,Score=0.51986038542,Split=('l',))
                    C(class=Iris-versicolor,Purity=1.0)
        I(Fidx=4,Score=0.691461352304,Split=('l',))
                C(class=Iris-setosa,Purity=0.5)
            I(Fidx=4,Score=0.678784588996,Split=('l',))
                        C(class=Iris-setosa,Purity=1.0)
                    I(Fidx=2,Score=0.277258872224,Split=('w',))
                        C(class=Iris-setosa,Purity=0.75)
                I(Fidx=4,Score=0.659325104991,Split=('l',))
        

What can you conclude ?
====================
Please write your observation....



# Cross-Validation

Until now we have been splitting the dataset into a training and test set rather randomly and were reporting a rather artifical performance. Now we are going to test our system exhaustively by making use of k-fold [cross validation](http://en.wikipedia.org/wiki/Cross-validation_%28statistics%29). 

Now go and tune your hyper-parameters (purity, exthreshold) to opitmize the performance.

In [59]:
# Now lets cross validate for best paramters, and test the result...
# We will be training four different models on four different partitions of data set and 
# then will be reporting the mean accuracy of the four classifiers.

nfolds=4 # lets use four folds..
folds=t.generate_folds(X,Y,nfolds)
features=arange(22)#[0, 1, 2, 3] # features to use for our system
#now lets train and test on these folds...
totacc=[]
for k in range(nfolds):
    dt=DecisionTree(0.95,5,7)
    dt.train(folds[k][0][:,features],folds[k][1])
    pclasses=dt.predict(folds[k][2][:,features])
    acc=np.sum(pclasses==folds[k][3])/float(folds[k][3].shape[0])
    print dt
    print "[Info] Fold {} Accuracy = {}".format(k+1, acc)
    totacc.append(acc)

print totacc, '\n Mean Accuracy =', np.mean(totacc)

Generating CV data for 2 classes


tools.py:472: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 8124 but corresponding boolean dimension is 149
  Xt = X[idx, :]  # get training examples


---------------------------------------------------
 A Decision Tree With Depth=8
            C(class=Iris-versicolor,Purity=1.0)
        I(Fidx=0,Score=0.247482631579,Split=('b',))
                        C(class=Iris-versicolor,Purity=1.0)
                    I(Fidx=5,Score=nan,Split=f)
                            C(class=Iris-versicolor,Purity=1.0)
                        I(Fidx=2,Score=3.68413614879e-15,Split=('n',))
                            C(class=Iris-setosa,Purity=0.5)
                I(Fidx=4,Score=0.115524530093,Split=('p',))
                    C(class=Iris-versicolor,Purity=1.0)
            I(Fidx=0,Score=0.280953899548,Split=('b',))
                        C(class=Iris-versicolor,Purity=1.0)
                    I(Fidx=0,Score=0.190954250488,Split=('b',))
                            C(class=Iris-setosa,Purity=1.0)
                        I(Fidx=0,Score=0.318257084147,Split=('b',))
                            C(class=Iris-versicolor,Purity=0.8)
                I(Fidx=0,Sc

In [60]:
# Now lets cross validate for best paramters, and test the result...
# We will be training four different models on four different partitions of data set and 
# then will be reporting the mean accuracy of the four classifiers.

nfolds=4 # lets use four folds..
folds=t.generate_folds(X,Y,nfolds)
features=[0,1, 2, 3] # features to use for our system
#now lets train and test on these folds...

#Lets perform the grid search...
purity=np.linspace(0.85,0.97,13)
nexamp=np.linspace(5,25,21)  

params=np.zeros((len(purity),len(nexamp)))
                   
for p in range(len(purity)):
    for n in range(len(nexamp)):
        totacc=[]
        for k in range(nfolds):
            dt=DecisionTree(purity[p],nexamp[n],7)#purity[p],nexamp[n])
            dt.train(folds[k][0][:,features],folds[k][1])
            pclasses=dt.predict(folds[k][2][:,features])
            acc=np.sum(pclasses==folds[k][3])/float(folds[k][3].shape[0])
            print "[Info] Fold {} Accuracy = {}".format(k+1, acc)
            totacc.append(acc)
        params[p,n]=np.mean(totacc)
        print totacc, '\nPurity={}, Nexample-threshold={}, Mean Accuracy ={}'.format(purity[p],nexamp[n], np.mean(totacc))

Generating CV data for 2 classes
[Info] Fold 1 Accuracy = 0.594594594595
[Info] Fold 2 Accuracy = 0.459459459459
[Info] Fold 3 Accuracy = 0.297297297297
[Info] Fold 4 Accuracy = 0.567567567568
[0.59459459459459463, 0.45945945945945948, 0.29729729729729731, 0.56756756756756754] 
Purity=0.85, Nexample-threshold=5.0, Mean Accuracy =0.47972972973
[Info] Fold 1 Accuracy = 0.594594594595
[Info] Fold 2 Accuracy = 0.459459459459
[Info] Fold 3 Accuracy = 0.297297297297
[Info] Fold 4 Accuracy = 0.567567567568
[0.59459459459459463, 0.45945945945945948, 0.29729729729729731, 0.56756756756756754] 
Purity=0.85, Nexample-threshold=6.0, Mean Accuracy =0.47972972973
[Info] Fold 1 Accuracy = 0.594594594595
[Info] Fold 2 Accuracy = 0.459459459459
[Info] Fold 3 Accuracy = 0.297297297297
[Info] Fold 4 Accuracy = 0.567567567568
[0.59459459459459463, 0.45945945945945948, 0.29729729729729731, 0.56756756756756754] 
Purity=0.85, Nexample-threshold=7.0, Mean Accuracy =0.47972972973
[Info] Fold 1 Accuracy = 0.5945

In [61]:
np.unravel_index(np.argmax(params), params.shape)

(0, 16)